# Hospitalizaciones COVID-19 en México

Basado en: https://github.com/RodrigoZepeda/CapacidadHospitalariaMX/

In [1]:
# Dependencies
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import os
from selenium import webdriver
from selenium.webdriver.support.select import Select
import time
import glob

#
import warnings
warnings.filterwarnings('ignore')

## Scraper de los datos de hospitalizaciones

In [2]:
# Configuración:

folder_of_download = "/Users/benjaminolivavazquez/Documents/Personal/SERVICIO SOCIAL/Paper-Covid-2020/Raw_Data"
direccion_chromedriver = 'Driver/chromedriver'
descargar_desde = "2021-02-12" #"2020-04-01"
descargar_hasta = "2021-01-03"
sleep_time    = 10 #Tiempo que tarda la página de la UNAM de cambiar ventana
download_time = 5  #Tiempo que tarda en descargarse el archivo en tu red

In [3]:
# Driver

option = webdriver.ChromeOptions()
option.add_argument("-incognito")

option.add_experimental_option("prefs", {
        "download.default_directory": folder_of_download,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing_for_trusted_sources_enabled": False,
        "safebrowsing.enabled": False
})

browser = webdriver.Chrome(executable_path = direccion_chromedriver, options = option)
browser.set_window_size(1000,1000)
browser.get("https://www.gits.igg.unam.mx/red-irag-dashboard/reviewHome#")

#Dar click en entrar
time.sleep(sleep_time)
browser.find_element_by_id('enter-button').click()

#Fechas a descargar
fechas_descargar = pd.date_range(start = descargar_desde, end = descargar_hasta, freq = 'D') 
fechas_descargar = fechas_descargar.strftime('%Y-%m-%d')
fechas_descargar = fechas_descargar[::-1]

#Impresión de tiempo estimado
print("Tiempo estimado: " + str(len(fechas_descargar)*(sleep_time*4 + 4*download_time)/360) + " horas")

#Elementos de resumen, ocupación general, camas y uci
resumen    = browser.find_element_by_xpath("/html/body/section/section[2]/div[1]/nav/ul/li[1]/a")
hospital   = browser.find_element_by_xpath("/html/body/section/section[2]/div[1]/nav/ul/li[4]/a")
ventilador = browser.find_element_by_xpath("/html/body/section/section[2]/div[1]/nav/ul/li[5]/a")
uci        = browser.find_element_by_xpath("/html/body/section/section[2]/div[1]/nav/ul/li[6]/a")

Tiempo estimado: 0.0 horas


In [4]:
#Avance para cuantificar
avance = 0

for fecha_analisis in fechas_descargar:
        
    print("Descargando " + str(100*avance/len(fechas_descargar)) + "%")
    avance = avance + 1
    
    #Asegurarnos que estamos en la sección resumen
    resumen.click()
    time.sleep(sleep_time)
        
    select = Select(browser.find_element_by_id("dateSelected"))
    select.select_by_visible_text(fecha_analisis)
    time.sleep(download_time)
    
    '''
    # --------------------------------
    # RESUMEN
    # --------------------------------
    
    #Unidades médicas que no reportaron dos días consecutivos
    unidades_dos = card_convert(browser,"/html/body/section/section[2]/div[2]/section/article[1]/div[2]/div[1]/div[1]/span[2]")
    
    #Unidades médicas que no reportaron este día
    unidades_este = card_convert(browser,"/html/body/section/section[2]/div[2]/section/article[1]/div[2]/div[1]/div[2]/span[2]")
    
    #Hospitalizado IRAG no UCI
    hosp_irag = card_convert(browser,"/html/body/section/section[2]/div[2]/section/article[1]/div[2]/div[1]/div[3]/span[2]")
    
    #Unidades con 70% hospitalización
    unidades_70hosp = card_convert(browser,"/html/body/section/section[2]/div[2]/section/article[1]/div[2]/div[2]/div/div[4]/div[1]/span[2]")
    
    #Unidades con 70% ventiladores
    unidades_70vent = card_convert(browser,"/html/body/section/section[2]/div[2]/section/article[1]/div[2]/div[2]/div/div[4]/div[2]/span[2]")
    
    #Unidades con 70% UCI
    unidades_70UCI = card_convert(browser,"/html/body/section/section[2]/div[2]/section/article[1]/div[2]/div[2]/div/div[4]/div[3]/span[2]")
    
    #Unidades entre 50 y 70 de hospitalización
    unidades_50hosp = card_convert(browser,"/html/body/section/section[2]/div[2]/section/article[1]/div[2]/div[2]/div/div[4]/div[4]/span[2]")
    
    #Unidades entre 50 y 70 de ventilador
    unidades_50vent = card_convert(browser,"/html/body/section/section[2]/div[2]/section/article[1]/div[2]/div[2]/div/div[4]/div[5]/span[2]")
    
    #Unidades entre 50 y 70 UCI
    unidades_50UCI  = card_convert(browser,"/html/body/section/section[2]/div[2]/section/article[1]/div[2]/div[2]/div/div[4]/div[6]/span[2]")
    '''     
    # --------------------------------
    # HOSPITALIZACIONES
    # --------------------------------
    
    hospital.click()
    time.sleep(sleep_time)
     
    browser.find_element_by_xpath("/html/body/section/section[2]/div[2]/section/article[2]/article/div[1]/div/div[5]/div/div[1]/button[1]").click()
    newname = "Hospitalizaciones_" + fecha_analisis + ".csv"
    time.sleep(download_time)
    os.rename(os.path.join(folder_of_download, "Sistema de Información de la Red IRAG.csv"), os.path.join(folder_of_download, newname))
    
    # --------------------------------
    # VENTILADORES
    # --------------------------------
    
    ventilador.click()
    time.sleep(sleep_time)

    browser.find_element_by_xpath("/html/body/section/section[2]/div[2]/section/article[2]/article/div[1]/div/div[5]/div/div[1]/button[1]").click()
    newname = "Ventiladores_" + fecha_analisis + ".csv"
    time.sleep(download_time)
    os.rename(os.path.join(folder_of_download, "Sistema de Información de la Red IRAG.csv"), os.path.join(folder_of_download, newname))
    
    # --------------------------------
    # UCI
    # --------------------------------
    
    uci.click()
    time.sleep(sleep_time)

    browser.find_element_by_xpath("/html/body/section/section[2]/div[2]/section/article[2]/article/div[1]/div/div[5]/div/div[1]/button[1]").click()
    newname = "UCI_" + fecha_analisis + ".csv"
    time.sleep(download_time)
    os.rename(os.path.join(folder_of_download, "Sistema de Información de la Red IRAG.csv"), os.path.join(folder_of_download, newname))
        
browser.close()

## Scraper de los datos de hospitalizaciones Estatales

In [8]:
# Configuración:

folder_of_download = "/Users/benjaminolivavazquez/Documents/Personal/SERVICIO SOCIAL/Paper-Covid-2020/Raw_Data_Estatal"
direccion_chromedriver = 'Driver/chromedriver'
descargar_desde = "2021-02-11" #"2020-04-01"
descargar_hasta = "2021-02-13"
sleep_time    = 10 #Tiempo que tarda la página de la UNAM de cambiar ventana
download_time = 5  #Tiempo que tarda en descargarse el archivo en tu red
estatal = True    #Poner como true si quieres datos estatales; como false si quieres clues

In [9]:
# 
option = webdriver.ChromeOptions()
option.add_argument("-incognito")

option.add_experimental_option("prefs", {
        "download.default_directory": folder_of_download,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing_for_trusted_sources_enabled": False,
        "safebrowsing.enabled": False
})

browser = webdriver.Chrome(executable_path=direccion_chromedriver, options=option)
browser.set_window_size(1000,1000)
browser.get("https://www.gits.igg.unam.mx/red-irag-dashboard/reviewHome#")

#Dar click en entrar
time.sleep(sleep_time)
browser.find_element_by_id('enter-button').click()

#Fechas a descargar
fechas_descargar = pd.date_range(start = descargar_desde, end = descargar_hasta, freq='D') 
fechas_descargar = fechas_descargar.strftime('%Y-%m-%d')
fechas_descargar = fechas_descargar[::-1]

#Impresión de tiempo estimado
print("Tiempo estimado: " + str(len(fechas_descargar)*(sleep_time*4 + 4*download_time)/360) + " horas")

#Elementos de resumen, ocupación general, camas y uci
resumen    = browser.find_element_by_xpath("/html/body/section/section[2]/div[1]/nav/ul/li[1]/a")
hospital   = browser.find_element_by_xpath("/html/body/section/section[2]/div[1]/nav/ul/li[4]/a")
ventilador = browser.find_element_by_xpath("/html/body/section/section[2]/div[1]/nav/ul/li[5]/a")
uci        = browser.find_element_by_xpath("/html/body/section/section[2]/div[1]/nav/ul/li[6]/a")

Tiempo estimado: 0.5 horas


In [10]:
#Avance para cuantificar
avance = 0

for fecha_analisis in fechas_descargar:
        
    print("Descargando " + str(100*avance/len(fechas_descargar)) + "%")
    avance = avance + 1
    
    #Asegurarnos que estamos en la sección resumen
    resumen.click()
    time.sleep(sleep_time)
        
    select = Select(browser.find_element_by_id("dateSelected"))
    select.select_by_visible_text(fecha_analisis)
    time.sleep(download_time)

    # --------------------------------
    # HOSPITALIZACIONES
    # --------------------------------
    
    hospital.click()
    time.sleep(sleep_time)
    
    #Cambiar el formato a estatal
    if estatal: 
        browser.find_element_by_xpath("/html/body/section/section[2]/div[2]/section/article[2]/article/div[1]/nav/ul/li[1]/a").click()
        time.sleep(sleep_time)        
        browser.find_element_by_xpath("/html/body/section/section[2]/div[2]/section/article[2]/article/div[1]/div/div[2]/div/div[1]/button[1]/span").click()
    else:
        browser.find_element_by_xpath("/html/body/section/section[2]/div[2]/section/article[2]/article/div[1]/div/div[5]/div/div[1]/button[1]/span").click()
    
    newname = "Hospitalizaciones_" + fecha_analisis + ".csv"
    time.sleep(download_time)
    os.rename(os.path.join(folder_of_download, "Sistema de Información de la Red IRAG.csv"), os.path.join(folder_of_download, newname))
    
    # --------------------------------
    # VENTILADORES
    # --------------------------------
    
    ventilador.click()
    time.sleep(sleep_time)

    if estatal: 
        browser.find_element_by_xpath("/html/body/section/section[2]/div[2]/section/article[2]/article/div[1]/nav/ul/li[1]/a").click()
        time.sleep(sleep_time)
        browser.find_element_by_xpath("/html/body/section/section[2]/div[2]/section/article[2]/article/div[1]/div/div[2]/div/div[1]/button[1]/span").click()
    else:     
        browser.find_element_by_xpath("/html/body/section/section[2]/div[2]/section/article[2]/article/div[1]/div/div[5]/div/div[1]/button[1]/span").click()
    newname = "Ventiladores_" + fecha_analisis + ".csv"
    time.sleep(download_time)
    os.rename(os.path.join(folder_of_download, "Sistema de Información de la Red IRAG.csv"), os.path.join(folder_of_download, newname))
    
    # --------------------------------
    # UCI
    # --------------------------------
    
    uci.click()
    time.sleep(sleep_time)
    
    if estatal: 
        browser.find_element_by_xpath("/html/body/section/section[2]/div[2]/section/article[2]/article/div[1]/nav/ul/li[1]/a").click()
        time.sleep(sleep_time)
        browser.find_element_by_xpath("/html/body/section/section[2]/div[2]/section/article[2]/article/div[1]/div/div[2]/div/div[1]/button[1]/span").click()   
    else:     
        browser.find_element_by_xpath("/html/body/section/section[2]/div[2]/section/article[2]/article/div[1]/div/div[5]/div/div[1]/button[1]/span").click()
    newname = "UCI_" + fecha_analisis + ".csv"
    time.sleep(download_time)
    os.rename(os.path.join(folder_of_download, "Sistema de Información de la Red IRAG.csv"), os.path.join(folder_of_download, newname))
        
browser.close()

Descargando 0.0%
Descargando 33.333333333333336%
Descargando 66.66666666666667%


## Hopitalizaciones a nivel NACIONAL

In [ ]:
# 
# Modificar diariamente
# 
Fecha = '10.03.2021'

df = pd.read_csv('http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2021/03/datos_abiertos_covid19_'+ Fecha +'.zip', 
                  compression = 'zip', encoding = 'latin1')

df.head()

## Base única

### Por CLUES

In [11]:
#
column_names = ['Unidad médica', 'Estado', 'Institución', 'CLUES', '% de Ocupación']

BU_CLUES = pd.DataFrame(columns = column_names)

#
path = "/Users/benjaminolivavazquez/Documents/Personal/SERVICIO SOCIAL/Paper-Covid-2020/Raw_Data/Hospitalizaciones_*.csv"

for fname in glob.glob(path):
    df1 = pd.read_csv(fname, encoding = 'utf-8-sig')
    df1['Fecha'] = fname[107:117]
    BU_CLUES = BU_CLUES.append( df1 )
#

In [12]:
# 
BU_CLUES['Date'] = pd.to_datetime( BU_CLUES['Fecha'] )

BU_CLUES = BU_CLUES.sort_values( by = ['Date'] )

BU_CLUES = BU_CLUES.reset_index()

BU_CLUES = BU_CLUES[['Unidad médica', 'Estado', 'Institución', 'CLUES', '% de Ocupación', 'Fecha', 'Date']]

BU_CLUES.head()

,Unidad médica,Estado,Institución,CLUES,% de Ocupación,Fecha,Date
0,Instituto Nacional de Rehabilitación,Ciudad de México,SSA/CC,DFSSA003956,100,2020-04-01,2020-04-01
1,Hospital Juárez de México,Ciudad de México,SSA/CC,DFSSA003944,100,2020-04-01,2020-04-01
2,Hospital Regional de Alta Especialidad Ixtapaluca,México,SSA/CC,MCSSA018786,100,2020-04-01,2020-04-01
3,Instituto Nacional de Enfermedades Respiratori...,Ciudad de México,SSA/CC,DFSSA004026,100,2020-04-01,2020-04-01
4,Instituto Nacional de Ciencias Médicas Y Nutri...,Ciudad de México,SSA/CC,DFSSA004055,100,2020-04-01,2020-04-01


### Por Estado

In [13]:
#
column_names = ['Estado', '% Ocupación']

BU_Estado = pd.DataFrame(columns = column_names)

#
path = "/Users/benjaminolivavazquez/Documents/Personal/SERVICIO SOCIAL/Paper-Covid-2020/Raw_Data_Estatal/Hospitalizaciones_*.csv"

for fname in glob.glob(path):
    df1 = pd.read_csv(fname, encoding = 'utf-8-sig')
    df1['Fecha'] = fname[115:125]
    BU_Estado = BU_Estado.append( df1 )
#

In [14]:
# Replace State Names
BU_Estado.loc[ BU_Estado['Estado'] == 'Veracruz de Ignacio de La Llave', 'Estado'] = 'Veracruz'
BU_Estado.loc[ BU_Estado['Estado'] == 'Coahuila de Zaragoza', 'Estado'] = 'Coahuila'
BU_Estado.loc[ BU_Estado['Estado'] == 'Michoacán de Ocampo', 'Estado'] = 'Michoacán'

In [15]:
# 
BU_Estado['Date'] = pd.to_datetime( BU_Estado['Fecha'] )

BU_Estado = BU_Estado.sort_values( by = ['Date'] )

BU_Estado = BU_Estado.reset_index()

BU_Estado = BU_Estado[['Estado', '% Ocupación', 'Fecha', 'Date']]

BU_Estado.head()

,Estado,% Ocupación,Fecha,Date
0,Oaxaca,100.0,2020-04-01,2020-04-01
1,Ciudad de México,100.0,2020-04-01,2020-04-01
2,México,100.0,2020-04-01,2020-04-01
3,Guanajuato,100.0,2020-04-01,2020-04-01
4,Oaxaca,100.0,2020-04-02,2020-04-02


## Visualizaciones por estados

In [22]:
# 
fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == 'Ciudad de México' ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == 'Ciudad de México' )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = 'Ciudad de México', line_color = 'darkblue' ) )

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == 'México' ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == 'México' )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = 'Estado de México', line_color = 'darkred' ) )

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == 'Jalisco' ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == 'Jalisco' )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = 'Jalisco', line_color = 'darkgreen' ) )

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == 'Nuevo León' ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == 'Nuevo León' )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = 'Nuevo León', line_color = 'orange' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': 'Ocupación de camas en hospitalización general',
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [27]:
# 1
ESTADO = 'Aguascalientes'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [28]:
# 2
ESTADO = 'Baja California'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [29]:
# 3
ESTADO = 'Baja California Sur'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [30]:
# 4
ESTADO = 'Campeche'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [31]:
# 5
ESTADO = 'Coahuila'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [32]:
# 6
ESTADO = 'Colima'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [33]:
# 7
ESTADO = 'Chiapas'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [34]:
# 8
ESTADO = 'Chihuahua'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [35]:
# 9
ESTADO = 'Ciudad de México'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [36]:
# 10
ESTADO = 'Durango'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [37]:
# 11
ESTADO = 'Guanajuato'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [38]:
# 12
ESTADO = 'Guerrero'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [39]:
# 13
ESTADO = 'Hidalgo'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [40]:
# 14
ESTADO = 'Jalisco'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [41]:
# 15
ESTADO = 'México'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [42]:
# 16
ESTADO = 'Michoacán'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [43]:
# 17
ESTADO = 'Morelos'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [44]:
# 18
ESTADO = 'Nayarit'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [45]:
# 19
ESTADO = 'Nuevo León'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [46]:
# 20
ESTADO = 'Oaxaca'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [47]:
# 21
ESTADO = 'Puebla'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [48]:
# 22
ESTADO = 'Querétaro'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [49]:
# 23
ESTADO = 'Quintana Roo'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [50]:
# 24
ESTADO = 'San Luis Potosí'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [51]:
# 25
ESTADO = 'Sinaloa'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [52]:
# 26
ESTADO = 'Sonora'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [53]:
# 27
ESTADO = 'Tabasco'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [54]:
# 28
ESTADO = 'Tamaulipas'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [55]:
# 29
ESTADO = 'Tlaxcala'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [56]:
# 30
ESTADO = 'Veracruz'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [57]:
# 31
ESTADO = 'Yucatán'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()

In [58]:
# 32
ESTADO = 'Zacatecas'

fig = go.Figure()

fig.add_trace( go.Scatter( x = BU_Estado[ ( BU_Estado['Estado'] == ESTADO ) \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['Date'], 
                          y = BU_Estado[ ( BU_Estado['Estado'] == ESTADO )  \
                                         & ( BU_Estado['Date'] > '2020-04-15' ) ]['% Ocupación'], 
                          name = ESTADO, line_color = 'darkblue' ) )

fig.update_layout( template = 'plotly_white', # "plotly", "plotly_white", "ggplot2", "seaborn", "simple_white", "none"
                  barmode = 'overlay', 
                  title = { 'text': ESTADO,
                           'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
                  font_family = 'Calibri Light')

fig.update_xaxes(tickformat = '%Y-%m-%d') # (tickangle = 90, tickformat = '%Y-%m-%d')

fig.show()